In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F


In [73]:
iris =load_iris()
X=iris['data']
y=iris['target']
names=iris['target_names']
feature_names=iris['feature_names']
# print(iris)
# print(names)
# print(feature_names)
scaler =StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split (
    X_scaled, y, test_size=0.2, random_state=2
)

In [74]:
# Create PyTorch Tensor datasets and dataloaders
train_dataset = TensorDataset(torch.Tensor(X_train), torch.LongTensor(y_train))
test_dataset = TensorDataset(torch.Tensor(X_test), torch.LongTensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [75]:
# Define the neural network model
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x


In [78]:
# Initialize the model
model = Model(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
# Train the model
EPOCHS = 400
for epoch in range(EPOCHS):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

# # Train the model using next in dataloader
# EPOCHS = 100
# data_iter = iter(train_loader)
# for epoch in range(EPOCHS):
#     for _ in range(len(train_loader)):
#         inputs, labels = next(data_iter)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         print(outputs, labels)
#         loss = loss_fn(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#     # Reset the dataloader at the end of each epoch
#     data_iter = iter(train_loader)

In [79]:
# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print(correct)

accuracy = correct / total
print(f'Accuracy: {accuracy}')

16
28
Accuracy: 0.9333333333333333
